In [21]:
import multiprocessing as mp
from pathlib import Path
import os
import json
import pandas as pd
from tqdm import tqdm
from lhotse import CutSet, RecordingSet, SupervisionSet, MonoCut
from lhotse.cut import append_cuts
from lhotse.recipes import prepare_voxpopuli, download_voxpopuli
import logging
from functools import reduce
from datasets import load_dataset, DownloadConfig
logging.basicConfig(level=logging.INFO)

In [8]:
# directory paths to save audio and transcript files
IN_DIR = "../datasets/LongSpeechSource/voxpopuli"
# directory paths to save metadata and processed aduio files
OUT_DIR = '../datasets/LongSpeech'

In [4]:
config = json.load(open(os.path.join(OUT_DIR, 'metadata.json')))
AVG_DURATION = config['avg_duration']
SAMPLE_RATE = config['sample_rate']
OUT_FILE_NAME = config['source']
prev_amount = config['amount']
print(prev_amount)

5290


In [5]:
# Download and prepare VoxPopuli dataset for ASR task in English
task = 'asr'
# Set to True to force re-download
ForceDownload = True  

if ForceDownload:
    download_voxpopuli(target_dir=IN_DIR, subset=task)



INFO:root:12 files to download...
100%|██████████| 5.29G/5.29G [04:12<00:00, 22.5MB/s]
  8%|▊         | 1/12 [05:38<1:01:58, 338.03s/it]

In [ ]:
manifests = prepare_voxpopuli(corpus_dir=IN_DIR, output_dir=OUT_DIR, task=task, lang=lang, num_jobs=15)

logging.info(f"RecordingSet: {manifests['train']['recordings']}")
logging.info(f"SupervisionSet: {manifests['train']['supervisions']}")

In [16]:
rs = manifests['train']['recordings']
ss = manifests['train']['supervisions']
cut = CutSet.from_manifests(recordings=rs, supervisions=ss)

In [12]:
cuts.to_jsonl(os.path.join(OUT_DIR, 'voxpopuli_raw_cuts.jsonl'))

In [8]:
"""
cuts = CutSet()
for part in manifests.keys():
    rs = manifests[part]['recordings']
    ss = manifests[part]['supervisions']
    cut = CutSet.from_manifests(recordings=rs, supervisions=ss)
    cuts += cut
"""

In [17]:
cuts = CutSet.from_jsonl(os.path.join(OUT_DIR, 'voxpopuli_raw_cuts.jsonl'))

In [25]:
cuts = cuts.sort_by_recording_id()

In [26]:
cuts.to_jsonl(os.path.join(OUT_DIR, 'cuts_sorted.jsonl'))